In [16]:
import sys
import os
sys.path.insert(0, '../src')

import torch
from torch import optim
import segmentation_models_pytorch as smp
import pytorch_lightning as pl

import matplotlib.pyplot as plt

In [3]:
from data_builder.builder import build_train_loader, build_valid_loader
from config import _C as cfg

In [4]:
data = os.listdir(os.path.join(cfg.DATA_DIR, 'image'))
train_lsit = list(set([row.split("_")[0] for row in data]))
train_idx = [row for row in data if row.split("_")[0] in train_lsit[:-2]]
valid_idx = [row for row in data if row.split("_")[0] not in train_lsit[:-2]]
len(train_idx),len(valid_idx)

(3028, 410)

In [5]:
train_loader = build_train_loader(cfg, train_idx)

In [6]:
valid_loader = build_valid_loader(cfg, valid_idx)

In [17]:
class hmapModel(pl.LightningModule):
    
    def __init__(self):
        super().__init__()
        self.model = smp.Unet('resnet34', encoder_weights='imagenet')
        
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = pl.metrics.functional.dice_score(y_hat, y)
        self.log('train_loss', loss)
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = pl.metrics.functional.dice_score(y_hat, y)
        self.log('valid_dice_coeff', loss)

In [18]:
model = hmapModel()

In [19]:
trainer = pl.Trainer()
trainer.fit(model, train_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
/home/giorgio/Scrivania/Kaggle/kaggleEnv/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: you defined a validation_step but have no val_dataloader. Skipping validation loop
  warnings.warn(*args, **kwargs)

  | Name  | Type | Params
-------------------------------
0 | model | Unet | 24 M  


RuntimeError: expected scalar type Byte but found Float